In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import re
import pandas as pd

In [2]:
# function to clear the books url (delete '/../../')

def clean_url(url):
    
    # Split the path by '/'
    path_parts = url.split('/')
    cleaned_path_parts = [part for part in path_parts if part and part != '..']

    # Join the cleaned path parts back together with '/'
    cleaned_link = '/'.join(cleaned_path_parts)

    return cleaned_link

In [3]:
# Initialize objects


# Init lists
Books_titles = []
Books_categories = []
Books_prices = []
Books_availability = []
Books_img = []
Books_Url = []
Books_descriptions = []
Books_stars = []


# Def categories
categories = [
    'travel',
    'children',
       'health',
    'psychology',
    'politics']

# Init url
base_url = "http://books.toscrape.com"
url = base_url

# open url
response = requests.get(url)

#-------------------------------------------------------------------------------
# loop into all category names
for category in categories :
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the category links in the main page
    category_select = soup.select(f'ul.nav-list li a[href*="catalogue/category/books/{category}"]')

    # Extract and print the URLs
    for cat in category_select:
        # find category url
        category_url = cat['href']
        next_url = base_url + '/' + category_url
        print(f"------------------------Category URL : {next_url}")

#-------------------------------------------------------------------------------

    # While Next button exists
    while next_url:
        # HTTP Request and BeautifulSoup
        r = requests.get(next_url)
        soup = BeautifulSoup(r.text)
        
        
        # find books and loop throught all of them
        for book in soup.find_all('article', class_='product_pod'):
            
            # find the title
            try:
                title = book.h3.a['title']
            except:
                title = None
                print('None - 1 ')
                

            
            
            # find the price
            try:
                price = book.find('p', class_='price_color').text
            except:
                price = None
                print('None - 2 ')
  
            
            # find the availability
            try:
                availability = book.find('p', class_='instock availability').text.strip()
            except:
                availability = None
                print('None - 3 ')

 
            # find the star rating
            try : 
                star_rating_tag = book.find('p', class_='star-rating')
                nb_star = star_rating_tag['class'][1]
            except:
                nb_star = None
                print('None - 4 ')
            
            
            # find the book url and nagivate in
            try :
                # book url
                extracted_link = book.h3.a['href']
                
                # create the url to get in
                book_url = base_url + '/catalogue/' + clean_url(extracted_link)
                
                # enter in the book page
                r2 = requests.get(book_url)
                soup2 = BeautifulSoup(r2.text, 'html.parser' )
                
                # find the descprition
                description_ = soup2.find('meta', {'name': 'description'})
                description = description_.get('content')

                # find the image link
                image_link = soup2.find('div', class_='item active').find('img')
                image_url = image_link['src']
            
            except :
                book_url = None
                description = None
                image_url = None
                print('None - 5 ')
                
                
            # store all informations in lists
                
            Books_categories.append(category)
            
            Books_titles.append(title)
            
            Books_prices.append(price)
                
            Books_availability.append(availability)
                
            Books_stars.append(nb_star)

            Books_Url.append(book_url)
            
            Books_descriptions.append(description)
            
            Books_img.append(image_url)

                
                



        try:
          next_tag = soup.find('li', class_='next').find('a')['href']
          next_url = urljoin(r.url, next_tag)
        except Exception as e:
            print(e)
            next_url = None
            
            
        sleep(0.2)
        
        print('Requesting next page : {}'.format(next_url) if next_url else 'Finished.')
        
        
        
# Store all the list in the dictionary
Storage = {"Books_titles" : Books_titles, 
           "Books_categories" : Books_categories,
           "Books_prices" : Books_prices,
           "Books_availability" : Books_availability,
           "Books_img" : Books_img,
           "Books_Url" : Books_Url,
           "Books_descriptions" : Books_descriptions,
           "Books_stars" : Books_stars
    
}


# create dataframe
df = pd.DataFrame(Storage)


# remove the '\n'
for col in df.columns:
    df[col] = df[col].replace('\n', ' ', regex=True)
    
    
# save the dataframe as a csv
df.to_csv('books_scraped.csv', sep=';')

------------------------Category URL : http://books.toscrape.com/catalogue/category/books/travel_2/index.html
'NoneType' object has no attribute 'find'
Finished.
------------------------Category URL : http://books.toscrape.com/catalogue/category/books/childrens_11/index.html
Requesting next page : http://books.toscrape.com/catalogue/category/books/childrens_11/page-2.html
'NoneType' object has no attribute 'find'
Finished.
------------------------Category URL : http://books.toscrape.com/catalogue/category/books/health_47/index.html
'NoneType' object has no attribute 'find'
Finished.
------------------------Category URL : http://books.toscrape.com/catalogue/category/books/psychology_26/index.html
'NoneType' object has no attribute 'find'
Finished.
------------------------Category URL : http://books.toscrape.com/catalogue/category/books/politics_48/index.html
'NoneType' object has no attribute 'find'
Finished.
